# experiment_3_k

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 553MB/s]


In [6]:
! unzip walmart-recruiting-store-sales-forecasting.zip

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [8]:
# Step 1: Setup and MLflow/DagsHub Configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Install required packages
!pip install prophet plotly mlflow dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.

In [9]:
# Import MLflow and DagsHub
import mlflow
import mlflow.sklearn
import dagshub
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
dagshub.init(repo_owner="konstantine25b", repo_name="Walmart-Recruiting---Store-Sales-Forecasting", mlflow=True)

# Set MLflow experiment
experiment_name = "Walmart_Prophet_Date_Features"
mlflow.set_experiment(experiment_name)

print("✅ MLflow and DagsHub initialized!")
print(f"🔬 Experiment: {experiment_name}")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b2aad0dc-1e32-45d1-b226-6b590404526b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=f6765cb66121ea77d6738d9cfe8fee03acb408b423ebf84be940d3b710fd39fb




Accessing as konstantine25b

Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

2025/06/25 08:02:26 INFO mlflow.tracking.fluent: Experiment with name 'Walmart_Prophet_Date_Features' does not exist. Creating a new experiment.


✅ MLflow and DagsHub initialized!
🔬 Experiment: Walmart_Prophet_Date_Features


In [11]:
# Step 2: Data Loading with MLflow Tracking
with mlflow.start_run(run_name="Data_Loading_Stores_Train_Only") as run:

    print(f"🔄 Starting data loading run: {run.info.run_id}")

    # Log parameters
    mlflow.log_param("data_sources", "stores.csv, train.csv")
    mlflow.log_param("features_csv_used", "No")
    mlflow.log_param("approach", "prophet_with_date_features")
    mlflow.log_param("branch", "simplified_stores_train")

    # Load data (only stores and train)
    stores = pd.read_csv('stores.csv')

    with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    train = pd.read_csv('train.csv')

    # Log data metrics
    mlflow.log_metric("train_samples", len(train))
    mlflow.log_metric("stores_count", len(stores))
    mlflow.log_metric("unique_stores_in_train", train['Store'].nunique())
    mlflow.log_metric("unique_departments", train['Dept'].nunique())
    mlflow.log_metric("date_range_days", (pd.to_datetime(train['Date'].max()) - pd.to_datetime(train['Date'].min())).days)

    # Basic data quality checks
    mlflow.log_metric("train_missing_values", train.isnull().sum().sum())
    mlflow.log_metric("stores_missing_values", stores.isnull().sum().sum())
    mlflow.log_metric("negative_sales_count", (train['Weekly_Sales'] < 0).sum())

    print("✅ Data loaded - Using only stores.csv and train.csv")
    print(f"📊 Train shape: {train.shape}")
    print(f"🏪 Stores shape: {stores.shape}")

🔄 Starting data loading run: c7b5171952eb49c9b00421fc6b62405f
✅ Data loaded - Using only stores.csv and train.csv
📊 Train shape: (421570, 5)
🏪 Stores shape: (45, 3)
🏃 View run Data_Loading_Stores_Train_Only at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/c7b5171952eb49c9b00421fc6b62405f
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2


In [12]:
# Step 3: Fixed Feature Engineering for 2-Year Dataset
with mlflow.start_run(run_name="Feature_Engineering_Date_Lagging_Fixed") as run:

    print(f"🔄 Starting feature engineering run: {run.info.run_id}")

    # Log parameters
    mlflow.log_param("feature_engineering_type", "comprehensive_date_and_lagging_2year_data")
    mlflow.log_param("data_period", "2010-02-05_to_2012-11-01")
    mlflow.log_param("lag_periods", "1,2,3,4,8,12")  # Removed 52-week lag
    mlflow.log_param("rolling_windows", "4,8,12,26")  # Max 26 weeks (6 months)
    mlflow.log_param("ewm_alphas", "0.1,0.3,0.5")

    def create_comprehensive_date_features(df):
        """Create extensive date features for retail forecasting"""
        df = df.copy()
        df['Date'] = pd.to_datetime(df['Date'])

        # Basic date components
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['DayOfYear'] = df['Date'].dt.dayofyear
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['Quarter'] = df['Date'].dt.quarter

        # Retail-specific time features
        df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        df['IsYearStart'] = df['Date'].dt.is_year_start.astype(int)
        df['IsYearEnd'] = df['Date'].dt.is_year_end.astype(int)

        # Cyclical encodings
        df['MonthSin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['MonthCos'] = np.cos(2 * np.pi * df['Month'] / 12)
        df['DayOfWeekSin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
        df['DayOfWeekCos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)
        df['DayOfYearSin'] = np.sin(2 * np.pi * df['DayOfYear'] / 365.25)
        df['DayOfYearCos'] = np.cos(2 * np.pi * df['DayOfYear'] / 365.25)
        df['WeekOfYearSin'] = np.sin(2 * np.pi * df['WeekOfYear'] / 52)
        df['WeekOfYearCos'] = np.cos(2 * np.pi * df['WeekOfYear'] / 52)

        # Time since reference
        reference_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - reference_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        df['MonthsFromStart'] = ((df['Date'].dt.year - reference_date.year) * 12 +
                                df['Date'].dt.month - reference_date.month)

        # Walmart-specific holiday features (based on provided dates)
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)
        df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)

        # Retail calendar features
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)  # Nov, Dec
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)    # Aug, Sep
        df['IsSummerSeason'] = df['Month'].isin([6, 7, 8]).astype(int) # Jun, Jul, Aug
        df['IsSpringSeaso'] = df['Month'].isin([3, 4, 5]).astype(int)  # Mar, Apr, May

        # Week patterns
        df['IsFirstWeekOfMonth'] = (df['Day'] <= 7).astype(int)
        df['IsLastWeekOfMonth'] = (df['Date'].dt.days_in_month - df['Day'] < 7).astype(int)

        return df

    def create_lagging_features_2year(df, target_col='Weekly_Sales'):
        """
        Create lagging features appropriate for 2-year dataset
        """
        df = df.copy()
        df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        # Reasonable lag features for 2-year data
        lags = [1, 2, 3, 4, 8, 12]  # Up to 12 weeks (3 months)

        for lag in lags:
            lag_col = f'{target_col}_lag_{lag}'
            df[lag_col] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)

        # Rolling window features (max 26 weeks = 6 months)
        windows = [4, 8, 12, 26]
        for window in windows:
            # Rolling mean
            df[f'{target_col}_rolling_mean_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .mean()
                .reset_index(level=[0,1], drop=True)
            )

            # Rolling std
            df[f'{target_col}_rolling_std_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .std()
                .reset_index(level=[0,1], drop=True)
            )

            # Rolling min/max
            df[f'{target_col}_rolling_min_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .min()
                .reset_index(level=[0,1], drop=True)
            )

            df[f'{target_col}_rolling_max_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .max()
                .reset_index(level=[0,1], drop=True)
            )

        # Exponential weighted features
        alphas = [0.1, 0.3, 0.5]
        for alpha in alphas:
            df[f'{target_col}_ewm_{alpha}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .ewm(alpha=alpha)
                .mean()
                .reset_index(level=[0,1], drop=True)
            )

        # Short-term differences only (no 52-week seasonal diff)
        df[f'{target_col}_trend_diff'] = (
            df.groupby(['Store', 'Dept'])[target_col].diff(1)   # Week-over-week
        )

        df[f'{target_col}_monthly_diff'] = (
            df.groupby(['Store', 'Dept'])[target_col].diff(4)   # Month-over-month (4 weeks)
        )

        df[f'{target_col}_quarterly_diff'] = (
            df.groupby(['Store', 'Dept'])[target_col].diff(12)  # Quarter-over-quarter (12 weeks)
        )

        # Year-over-year features (for data we have)
        # Only calculate if we have at least 1 year of data
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['YearWeek'] = df['Date'].dt.year.astype(str) + '_' + df['WeekOfYear'].astype(str).str.zfill(2)

        # Same week previous year (only for 2011 and 2012 data)
        df[f'{target_col}_same_week_prev_year'] = (
            df.groupby(['Store', 'Dept', 'WeekOfYear'])[target_col].shift(1)
        )

        # Holiday interaction features
        df[f'{target_col}_pre_holiday'] = df.groupby(['Store', 'Dept'])['IsHoliday'].shift(-1).fillna(0).astype(int)
        df[f'{target_col}_post_holiday'] = df.groupby(['Store', 'Dept'])['IsHoliday'].shift(1).fillna(0).astype(int)

        return df

    # Apply feature engineering
    initial_cols = len(train.columns)

    train_merged = train.merge(stores, on='Store', how='left')
    mlflow.log_metric("after_store_merge_cols", len(train_merged.columns))

    train_with_features = create_comprehensive_date_features(train_merged)
    date_features_added = len(train_with_features.columns) - len(train_merged.columns)
    mlflow.log_metric("date_features_added", date_features_added)

    train_final = create_lagging_features_2year(train_with_features)
    lag_features_added = len(train_final.columns) - len(train_with_features.columns)
    mlflow.log_metric("lag_features_added", lag_features_added)

    total_features_added = len(train_final.columns) - initial_cols
    mlflow.log_metric("total_features_added", total_features_added)
    mlflow.log_metric("final_dataset_cols", len(train_final.columns))
    mlflow.log_metric("final_dataset_rows", len(train_final))

    # Data quality after feature engineering
    mlflow.log_metric("missing_values_after_features", train_final.isnull().sum().sum())
    mlflow.log_metric("infinite_values", np.isinf(train_final.select_dtypes(include=[np.number])).sum().sum())

    # Log data coverage analysis
    date_range = pd.to_datetime(train_final['Date'].max()) - pd.to_datetime(train_final['Date'].min())
    mlflow.log_metric("total_weeks_in_data", date_range.days // 7)
    mlflow.log_metric("years_of_data", date_range.days / 365.25)

    # Holiday coverage analysis
    holiday_weeks = train_final['IsHoliday'].sum()
    major_holiday_weeks = train_final['IsMajorHoliday'].sum()
    mlflow.log_metric("holiday_weeks_total", holiday_weeks)
    mlflow.log_metric("major_holiday_weeks", major_holiday_weeks)

    print(f"✅ Feature engineering completed for 2-year dataset!")
    print(f"📊 Final dataset shape: {train_final.shape}")
    print(f"📋 Features added: {total_features_added}")
    print(f"📅 Data period: {train_final['Date'].min()} to {train_final['Date'].max()}")
    print(f"🎯 Holiday weeks identified: {holiday_weeks} total, {major_holiday_weeks} major holidays")

    # Show feature summary
    feature_categories = {
        'Date Features': [col for col in train_final.columns if any(x in col for x in ['Year', 'Month', 'Day', 'Week', 'Quarter', 'Sin', 'Cos'])],
        'Holiday Features': [col for col in train_final.columns if any(x in col for x in ['Holiday', 'SuperBowl', 'Labor', 'Thanksgiving', 'Christmas'])],
        'Lag Features': [col for col in train_final.columns if 'lag_' in col],
        'Rolling Features': [col for col in train_final.columns if 'rolling_' in col],
        'EWM Features': [col for col in train_final.columns if 'ewm_' in col],
        'Diff Features': [col for col in train_final.columns if 'diff' in col]
    }

    for category, features in feature_categories.items():
        mlflow.log_metric(f"{category.lower().replace(' ', '_')}_count", len(features))
        if features:
            print(f"\n{category}: {len(features)} features")
            print(f"  Examples: {features[:3]}")

🔄 Starting feature engineering run: cec4a44589a94079ae5a64e8662006ae
✅ Feature engineering completed for 2-year dataset!
📊 Final dataset shape: (421570, 75)
📋 Features added: 70
📅 Data period: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🎯 Holiday weeks identified: 29661 total, 29661 major holidays

Date Features: 65 features
  Examples: ['Weekly_Sales', 'Year', 'Month']

Holiday Features: 7 features
  Examples: ['IsHoliday', 'IsSuperBowlWeek', 'IsLaborDayWeek']

Lag Features: 6 features
  Examples: ['Weekly_Sales_lag_1', 'Weekly_Sales_lag_2', 'Weekly_Sales_lag_3']

Rolling Features: 16 features
  Examples: ['Weekly_Sales_rolling_mean_4', 'Weekly_Sales_rolling_std_4', 'Weekly_Sales_rolling_min_4']

EWM Features: 3 features
  Examples: ['Weekly_Sales_ewm_0.1', 'Weekly_Sales_ewm_0.3', 'Weekly_Sales_ewm_0.5']

Diff Features: 3 features
  Examples: ['Weekly_Sales_trend_diff', 'Weekly_Sales_monthly_diff', 'Weekly_Sales_quarterly_diff']
🏃 View run Feature_Engineering_Date_Lagging_Fixed at: htt

In [13]:
# Step: NaN Analysis and Data Quality Check
with mlflow.start_run(run_name="Data_Quality_NaN_Analysis") as run:

    print(f"🔍 Starting NaN analysis run: {run.info.run_id}")

    mlflow.log_param("analysis_type", "comprehensive_nan_check")
    mlflow.log_param("dataset_shape", f"{train_final.shape[0]}x{train_final.shape[1]}")

    # 1. Overall NaN Summary
    total_cells = train_final.shape[0] * train_final.shape[1]
    total_nans = train_final.isnull().sum().sum()
    nan_percentage = (total_nans / total_cells) * 100

    mlflow.log_metric("total_cells", total_cells)
    mlflow.log_metric("total_nans", total_nans)
    mlflow.log_metric("nan_percentage", nan_percentage)

    print(f"📊 Overall NaN Summary:")
    print(f"   Total cells: {total_cells:,}")
    print(f"   Total NaNs: {total_nans:,}")
    print(f"   NaN percentage: {nan_percentage:.2f}%")

    # 2. NaN by Column
    nan_by_column = train_final.isnull().sum()
    nan_columns = nan_by_column[nan_by_column > 0].sort_values(ascending=False)

    mlflow.log_metric("columns_with_nans", len(nan_columns))
    mlflow.log_metric("columns_without_nans", len(train_final.columns) - len(nan_columns))

    print(f"\n📋 Columns with NaNs: {len(nan_columns)} out of {len(train_final.columns)}")

    if len(nan_columns) > 0:
        print(f"\n🔍 Detailed NaN Analysis by Column:")
        print("="*60)

        nan_analysis = pd.DataFrame({
            'Column': nan_columns.index,
            'NaN_Count': nan_columns.values,
            'NaN_Percentage': (nan_columns.values / len(train_final)) * 100,
            'Data_Type': [str(train_final[col].dtype) for col in nan_columns.index]
        }).reset_index(drop=True)

        print(nan_analysis.to_string(index=False))

        # Log top NaN columns
        top_nan_columns = nan_analysis.head(10)
        for i, row in top_nan_columns.iterrows():
            mlflow.log_metric(f"nan_count_{row['Column']}", row['NaN_Count'])
            mlflow.log_metric(f"nan_pct_{row['Column']}", row['NaN_Percentage'])

        # Save detailed analysis
        nan_analysis.to_csv("nan_analysis_detailed.csv", index=False)
        mlflow.log_artifact("nan_analysis_detailed.csv")

    # 3. NaN by Feature Category
    feature_categories = {
        'Original Features': ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Type', 'Size'],
        'Date Features': [col for col in train_final.columns if any(x in col for x in ['Year', 'Month', 'Day', 'Week', 'Quarter', 'Sin', 'Cos', 'DaysFrom', 'MonthsFrom'])],
        'Holiday Features': [col for col in train_final.columns if any(x in col for x in ['Holiday', 'SuperBowl', 'Labor', 'Thanksgiving', 'Christmas']) and col != 'IsHoliday'],
        'Lag Features': [col for col in train_final.columns if 'lag_' in col],
        'Rolling Features': [col for col in train_final.columns if 'rolling_' in col],
        'EWM Features': [col for col in train_final.columns if 'ewm_' in col],
        'Diff Features': [col for col in train_final.columns if 'diff' in col or 'same_week_prev_year' in col]
    }

    print(f"\n📈 NaN Analysis by Feature Category:")
    print("="*60)

    for category, features in feature_categories.items():
        category_features = [col for col in features if col in train_final.columns]
        if category_features:
            category_nans = train_final[category_features].isnull().sum().sum()
            category_cells = len(train_final) * len(category_features)
            category_nan_pct = (category_nans / category_cells) * 100 if category_cells > 0 else 0

            mlflow.log_metric(f"{category.lower().replace(' ', '_')}_nans", category_nans)
            mlflow.log_metric(f"{category.lower().replace(' ', '_')}_nan_pct", category_nan_pct)

            print(f"{category}:")
            print(f"   Features: {len(category_features)}")
            print(f"   NaNs: {category_nans:,} ({category_nan_pct:.2f}%)")

            # Show which specific features have NaNs
            cat_nan_features = train_final[category_features].isnull().sum()
            cat_nan_features = cat_nan_features[cat_nan_features > 0]
            if len(cat_nan_features) > 0:
                print(f"   Features with NaNs: {list(cat_nan_features.index)}")
            print()

    # 4. NaN Pattern Analysis
    print(f"\n🔍 NaN Pattern Analysis:")
    print("="*40)

    # Check if NaNs follow expected patterns for lagging features
    lag_features = [col for col in train_final.columns if 'lag_' in col]
    if lag_features:
        print(f"Lag Features NaN Analysis:")
        for lag_col in lag_features:
            lag_num = int(lag_col.split('_lag_')[1])
            expected_nans_per_group = lag_num

            # Count NaNs by Store-Dept group
            group_nan_counts = train_final.groupby(['Store', 'Dept'])[lag_col].apply(lambda x: x.isnull().sum())
            actual_nans = train_final[lag_col].isnull().sum()

            print(f"   {lag_col}: {actual_nans:,} NaNs")
            mlflow.log_metric(f"nans_{lag_col}", actual_nans)

    # Check rolling features
    rolling_features = [col for col in train_final.columns if 'rolling_' in col]
    if rolling_features:
        print(f"\nRolling Features NaN Analysis:")
        for roll_col in rolling_features[:3]:  # Show first 3
            nans = train_final[roll_col].isnull().sum()
            print(f"   {roll_col}: {nans:,} NaNs")

    # Check diff features
    diff_features = [col for col in train_final.columns if 'diff' in col]
    if diff_features:
        print(f"\nDiff Features NaN Analysis:")
        for diff_col in diff_features:
            nans = train_final[diff_col].isnull().sum()
            print(f"   {diff_col}: {nans:,} NaNs")

    # 5. Sample of rows with most NaNs
    print(f"\n🎯 Sample Analysis:")
    print("="*30)

    # Count NaNs per row
    nans_per_row = train_final.isnull().sum(axis=1)
    max_nans_per_row = nans_per_row.max()
    rows_with_max_nans = (nans_per_row == max_nans_per_row).sum()

    mlflow.log_metric("max_nans_per_row", max_nans_per_row)
    mlflow.log_metric("rows_with_max_nans", rows_with_max_nans)

    print(f"Maximum NaNs in single row: {max_nans_per_row}")
    print(f"Rows with maximum NaNs: {rows_with_max_nans:,}")

    # Show distribution of NaNs per row
    nan_distribution = nans_per_row.value_counts().sort_index()
    print(f"\nDistribution of NaNs per row:")
    for nan_count, row_count in nan_distribution.head(10).items():
        print(f"   {nan_count} NaNs: {row_count:,} rows")

    # 6. Show sample of problematic rows
    if max_nans_per_row > 0:
        print(f"\n📋 Sample of rows with most NaNs:")
        sample_problematic = train_final[nans_per_row == max_nans_per_row].head(3)
        basic_info = sample_problematic[['Store', 'Dept', 'Date', 'Weekly_Sales']].copy()
        print(basic_info.to_string(index=False))

    # 7. Recommendations
    print(f"\n💡 Recommendations:")
    print("="*20)

    if total_nans == 0:
        print("✅ No NaNs found! Dataset is clean.")
        mlflow.log_param("recommendation", "no_action_needed")
    else:
        if nan_percentage < 1:
            print("✅ Low NaN percentage (<1%) - mostly expected from lagging features")
            mlflow.log_param("recommendation", "minimal_cleaning_needed")
        elif nan_percentage < 5:
            print("⚠️  Moderate NaN percentage (1-5%) - review and handle appropriately")
            mlflow.log_param("recommendation", "moderate_cleaning_needed")
        else:
            print("🚨 High NaN percentage (>5%) - significant cleaning required")
            mlflow.log_param("recommendation", "extensive_cleaning_needed")

        print(f"\nSuggested actions:")
        print(f"1. Lag features: NaNs are expected - consider forward fill or drop early rows")
        print(f"2. Rolling features: Should have minimal NaNs due to min_periods=1")
        print(f"3. Diff features: NaNs expected for first observations")
        print(f"4. Consider Prophet's built-in missing value handling")

    print(f"\n🏃 Analysis complete! Check MLflow for detailed metrics.")

🔍 Starting NaN analysis run: 9835ab3010f64015a3c7afaa25d3071f
📊 Overall NaN Summary:
   Total cells: 31,617,750
   Total NaNs: 325,131
   NaN percentage: 1.03%

📋 Columns with NaNs: 14 out of 75

🔍 Detailed NaN Analysis by Column:
                          Column  NaN_Count  NaN_Percentage Data_Type
Weekly_Sales_same_week_prev_year     159167       37.755770   float64
             Weekly_Sales_lag_12      38615        9.159807   float64
     Weekly_Sales_quarterly_diff      38615        9.159807   float64
              Weekly_Sales_lag_8      25966        6.159357   float64
              Weekly_Sales_lag_4      13134        3.115497   float64
       Weekly_Sales_monthly_diff      13134        3.115497   float64
              Weekly_Sales_lag_3       9889        2.345755   float64
              Weekly_Sales_lag_2       6625        1.571507   float64
              Weekly_Sales_lag_1       3331        0.790142   float64
      Weekly_Sales_rolling_std_4       3331        0.790142   float64

In [14]:
# Step: Fixed Feature Engineering - Handle NaNs Properly
with mlflow.start_run(run_name="Feature_Engineering_NaN_Fixed") as run:

    print(f"🔧 Starting NaN-fixed feature engineering: {run.info.run_id}")

    mlflow.log_param("feature_engineering_type", "nan_fixed_comprehensive")
    mlflow.log_param("nan_handling_strategy", "remove_problematic_fill_expected")
    mlflow.log_param("removed_features", "Weekly_Sales_same_week_prev_year")

    def create_comprehensive_date_features(df):
        """Create extensive date features for retail forecasting"""
        df = df.copy()
        df['Date'] = pd.to_datetime(df['Date'])

        # Basic date components
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['DayOfYear'] = df['Date'].dt.dayofyear
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['Quarter'] = df['Date'].dt.quarter

        # Retail-specific time features
        df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        df['IsYearStart'] = df['Date'].dt.is_year_start.astype(int)
        df['IsYearEnd'] = df['Date'].dt.is_year_end.astype(int)

        # Cyclical encodings
        df['MonthSin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['MonthCos'] = np.cos(2 * np.pi * df['Month'] / 12)
        df['DayOfWeekSin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
        df['DayOfWeekCos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)
        df['DayOfYearSin'] = np.sin(2 * np.pi * df['DayOfYear'] / 365.25)
        df['DayOfYearCos'] = np.cos(2 * np.pi * df['DayOfYear'] / 365.25)
        df['WeekOfYearSin'] = np.sin(2 * np.pi * df['WeekOfYear'] / 52)
        df['WeekOfYearCos'] = np.cos(2 * np.pi * df['WeekOfYear'] / 52)

        # Time since reference
        reference_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - reference_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        df['MonthsFromStart'] = ((df['Date'].dt.year - reference_date.year) * 12 +
                                df['Date'].dt.month - reference_date.month)

        # Walmart-specific holiday features
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)
        df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)

        # Retail calendar features
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)
        df['IsSummerSeason'] = df['Month'].isin([6, 7, 8]).astype(int)
        df['IsSpringSeaso'] = df['Month'].isin([3, 4, 5]).astype(int)

        # Week patterns
        df['IsFirstWeekOfMonth'] = (df['Day'] <= 7).astype(int)
        df['IsLastWeekOfMonth'] = (df['Date'].dt.days_in_month - df['Day'] < 7).astype(int)

        return df

    def create_lagging_features_nan_fixed(df, target_col='Weekly_Sales'):
        """
        Create lagging features with proper NaN handling
        """
        df = df.copy()
        df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        # Lag features (expected NaNs at the beginning)
        lags = [1, 2, 3, 4, 8, 12]

        for lag in lags:
            lag_col = f'{target_col}_lag_{lag}'
            df[lag_col] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)

            # Forward fill NaNs (use the first available value)
            df[lag_col] = df.groupby(['Store', 'Dept'])[lag_col].fillna(method='bfill')

        # Rolling window features
        windows = [4, 8, 12, 26]
        for window in windows:
            # Rolling mean (min_periods=1 prevents NaNs)
            df[f'{target_col}_rolling_mean_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .mean()
                .reset_index(level=[0,1], drop=True)
            )

            # Rolling std (min_periods=2 to avoid NaNs when only 1 value)
            df[f'{target_col}_rolling_std_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=2)
                .std()
                .reset_index(level=[0,1], drop=True)
            )

            # Fill remaining NaNs in std with 0 (indicates no volatility)
            df[f'{target_col}_rolling_std_{window}'] = df[f'{target_col}_rolling_std_{window}'].fillna(0)

            # Rolling min/max (min_periods=1)
            df[f'{target_col}_rolling_min_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .min()
                .reset_index(level=[0,1], drop=True)
            )

            df[f'{target_col}_rolling_max_{window}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1)
                .max()
                .reset_index(level=[0,1], drop=True)
            )

        # Exponential weighted features (no NaNs expected)
        alphas = [0.1, 0.3, 0.5]
        for alpha in alphas:
            df[f'{target_col}_ewm_{alpha}'] = (
                df.groupby(['Store', 'Dept'])[target_col]
                .ewm(alpha=alpha)
                .mean()
                .reset_index(level=[0,1], drop=True)
            )

        # Difference features with proper NaN handling
        # Week-over-week difference
        df[f'{target_col}_trend_diff'] = (
            df.groupby(['Store', 'Dept'])[target_col].diff(1)
        )
        # Fill first observation with 0 (no change from "previous")
        df[f'{target_col}_trend_diff'] = df.groupby(['Store', 'Dept'])[f'{target_col}_trend_diff'].fillna(0)

        # Month-over-month difference (4 weeks)
        df[f'{target_col}_monthly_diff'] = (
            df.groupby(['Store', 'Dept'])[target_col].diff(4)
        )
        # Fill with week-over-week difference for first 4 observations
        mask = df[f'{target_col}_monthly_diff'].isnull()
        df.loc[mask, f'{target_col}_monthly_diff'] = df.loc[mask, f'{target_col}_trend_diff']

        # Quarter-over-quarter difference (12 weeks)
        df[f'{target_col}_quarterly_diff'] = (
            df.groupby(['Store', 'Dept'])[target_col].diff(12)
        )
        # Fill with monthly difference for first 12 observations
        mask = df[f'{target_col}_quarterly_diff'].isnull()
        df.loc[mask, f'{target_col}_quarterly_diff'] = df.loc[mask, f'{target_col}_monthly_diff']

        # REMOVED: Weekly_Sales_same_week_prev_year (too many NaNs)
        # This was causing 37.8% NaNs which is too much

        # Holiday interaction features
        df[f'{target_col}_pre_holiday'] = df.groupby(['Store', 'Dept'])['IsHoliday'].shift(-1).fillna(0).astype(int)
        df[f'{target_col}_post_holiday'] = df.groupby(['Store', 'Dept'])['IsHoliday'].shift(1).fillna(0).astype(int)

        return df

    # Apply feature engineering
    initial_cols = len(train.columns)

    train_merged = train.merge(stores, on='Store', how='left')
    mlflow.log_metric("after_store_merge_cols", len(train_merged.columns))

    train_with_features = create_comprehensive_date_features(train_merged)
    date_features_added = len(train_with_features.columns) - len(train_merged.columns)
    mlflow.log_metric("date_features_added", date_features_added)

    train_final_fixed = create_lagging_features_nan_fixed(train_with_features)
    lag_features_added = len(train_final_fixed.columns) - len(train_with_features.columns)
    mlflow.log_metric("lag_features_added", lag_features_added)

    total_features_added = len(train_final_fixed.columns) - initial_cols
    mlflow.log_metric("total_features_added", total_features_added)
    mlflow.log_metric("final_dataset_cols", len(train_final_fixed.columns))
    mlflow.log_metric("final_dataset_rows", len(train_final_fixed))

    # Check NaNs after fixes
    total_nans_after = train_final_fixed.isnull().sum().sum()
    mlflow.log_metric("total_nans_after_fix", total_nans_after)
    mlflow.log_metric("nan_percentage_after_fix", (total_nans_after / (train_final_fixed.shape[0] * train_final_fixed.shape[1])) * 100)

    print(f"✅ Feature engineering completed with NaN fixes!")
    print(f"📊 Final dataset shape: {train_final_fixed.shape}")
    print(f"📋 Features added: {total_features_added}")
    print(f"🔧 NaNs after fixes: {total_nans_after:,}")

    # Verify no NaNs remain
    if total_nans_after == 0:
        print("🎉 SUCCESS: No NaNs remaining in dataset!")
        mlflow.log_param("nan_fix_status", "complete_success")
    else:
        print(f"⚠️  Warning: {total_nans_after:,} NaNs still remain")
        mlflow.log_param("nan_fix_status", "partial_success")

        # Show remaining NaN columns
        remaining_nans = train_final_fixed.isnull().sum()
        remaining_nans = remaining_nans[remaining_nans > 0]
        if len(remaining_nans) > 0:
            print("Columns with remaining NaNs:")
            for col, count in remaining_nans.items():
                print(f"   {col}: {count:,} NaNs")

    # Quick verification of key feature types
    feature_verification = {
        'Lag Features': len([col for col in train_final_fixed.columns if 'lag_' in col]),
        'Rolling Features': len([col for col in train_final_fixed.columns if 'rolling_' in col]),
        'EWM Features': len([col for col in train_final_fixed.columns if 'ewm_' in col]),
        'Diff Features': len([col for col in train_final_fixed.columns if 'diff' in col]),
        'Holiday Features': len([col for col in train_final_fixed.columns if any(x in col for x in ['Holiday', 'SuperBowl', 'Labor', 'Thanksgiving', 'Christmas'])])
    }

    print(f"\n📋 Feature Verification:")
    for feature_type, count in feature_verification.items():
        print(f"   {feature_type}: {count}")
        mlflow.log_metric(f"{feature_type.lower().replace(' ', '_')}_count_final", count)

🔧 Starting NaN-fixed feature engineering: b340ff13b5684dc98dbb82de5b61f914
✅ Feature engineering completed with NaN fixes!
📊 Final dataset shape: (421570, 73)
📋 Features added: 68
🔧 NaNs after fixes: 2,035
⚠️  Warning: 2,035 NaNs still remain
Columns with remaining NaNs:
   Weekly_Sales_lag_1: 37 NaNs
   Weekly_Sales_lag_2: 97 NaNs
   Weekly_Sales_lag_3: 154 NaNs
   Weekly_Sales_lag_4: 226 NaNs
   Weekly_Sales_lag_8: 526 NaNs
   Weekly_Sales_lag_12: 995 NaNs

📋 Feature Verification:
   Lag Features: 6
   Rolling Features: 16
   EWM Features: 3
   Diff Features: 3
   Holiday Features: 7
🏃 View run Feature_Engineering_NaN_Fixed at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/b340ff13b5684dc98dbb82de5b61f914
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2


In [17]:
# Step: Final NaN Cleanup for Lag Features
with mlflow.start_run(run_name="Final_NaN_Cleanup_Lag_Features") as run:

    print(f"🔧 Starting final NaN cleanup: {run.info.run_id}")

    mlflow.log_param("cleanup_strategy", "fill_lag_nans_with_group_statistics")
    mlflow.log_param("remaining_nans_before", 2035)

    # Create a copy for final cleanup
    train_final_clean = train_final_fixed.copy()

    # Strategy: Fill lag feature NaNs with group-specific statistics
    lag_features = [col for col in train_final_clean.columns if 'lag_' in col]

    print(f"🔍 Cleaning {len(lag_features)} lag features...")

    for lag_col in lag_features:
        nans_before = train_final_clean[lag_col].isnull().sum()

        if nans_before > 0:
            print(f"   Fixing {lag_col}: {nans_before} NaNs")

            # Method 1: Forward fill within each Store-Dept group first
            train_final_clean[lag_col] = train_final_clean.groupby(['Store', 'Dept'])[lag_col].fillna(method='ffill')

            # Method 2: For remaining NaNs, use the group's mean Weekly_Sales
            # This handles store-dept combinations with insufficient history
            mask = train_final_clean[lag_col].isnull()
            if mask.sum() > 0:
                group_means = train_final_clean.groupby(['Store', 'Dept'])['Weekly_Sales'].transform('mean')
                train_final_clean.loc[mask, lag_col] = group_means[mask]

            # Method 3: For any remaining NaNs (shouldn't happen), use overall group statistics
            if train_final_clean[lag_col].isnull().sum() > 0:
                # Use store-level mean if dept-level isn't available
                store_means = train_final_clean.groupby('Store')['Weekly_Sales'].transform('mean')
                mask = train_final_clean[lag_col].isnull()
                train_final_clean.loc[mask, lag_col] = store_means[mask]

            # Method 4: Ultimate fallback - use department-level mean across all stores
            if train_final_clean[lag_col].isnull().sum() > 0:
                dept_means = train_final_clean.groupby('Dept')['Weekly_Sales'].transform('mean')
                mask = train_final_clean[lag_col].isnull()
                train_final_clean.loc[mask, lag_col] = dept_means[mask]

            nans_after = train_final_clean[lag_col].isnull().sum()
            print(f"      Before: {nans_before}, After: {nans_after}")
            mlflow.log_metric(f"nans_fixed_{lag_col}", nans_before - nans_after)

    # Final verification
    total_nans_final = train_final_clean.isnull().sum().sum()
    mlflow.log_metric("total_nans_final", total_nans_final)
    mlflow.log_metric("final_nan_percentage", (total_nans_final / (train_final_clean.shape[0] * train_final_clean.shape[1])) * 100)

    print(f"\n✅ Final cleanup completed!")
    print(f"📊 Dataset shape: {train_final_clean.shape}")
    print(f"🔧 Total NaNs remaining: {total_nans_final}")

    if total_nans_final == 0:
        print("🎉 SUCCESS: Dataset is now completely clean with 0 NaNs!")
        mlflow.log_param("final_status", "completely_clean")
    else:
        print(f"⚠️  {total_nans_final} NaNs still remain after all cleanup attempts")
        mlflow.log_param("final_status", "needs_investigation")

        # Show any remaining problematic columns
        remaining_nans = train_final_clean.isnull().sum()
        remaining_nans = remaining_nans[remaining_nans > 0]
        if len(remaining_nans) > 0:
            print("Remaining NaN columns:")
            for col, count in remaining_nans.items():
                print(f"   {col}: {count}")

    # Data quality summary
    print(f"\n📋 Final Dataset Summary:")
    print(f"   Shape: {train_final_clean.shape}")
    print(f"   Date range: {train_final_clean['Date'].min()} to {train_final_clean['Date'].max()}")
    print(f"   Stores: {train_final_clean['Store'].nunique()}")
    print(f"   Departments: {train_final_clean['Dept'].nunique()}")
    print(f"   Store-Dept combinations: {train_final_clean.groupby(['Store', 'Dept']).ngroups}")
    print(f"   Average weeks per combination: {len(train_final_clean) / train_final_clean.groupby(['Store', 'Dept']).ngroups:.1f}")

    # Log final dataset metrics
    mlflow.log_metric("final_shape_rows", train_final_clean.shape[0])
    mlflow.log_metric("final_shape_cols", train_final_clean.shape[1])
    mlflow.log_metric("final_stores", train_final_clean['Store'].nunique())
    mlflow.log_metric("final_departments", train_final_clean['Dept'].nunique())
    mlflow.log_metric("final_store_dept_combinations", train_final_clean.groupby(['Store', 'Dept']).ngroups)

    print(f"\n🎯 Dataset is ready for Prophet training!")

# Now run the quality verification
with mlflow.start_run(run_name="Data_Quality_Final_Verification") as run:

    print(f"🔍 Final data quality check: {run.info.run_id}")

    # Check for any data quality issues
    quality_checks = {}

    # 1. Check for NaNs
    total_nans = train_final_clean.isnull().sum().sum()
    quality_checks['total_nans'] = total_nans

    # 2. Check for infinite values
    numeric_cols = train_final_clean.select_dtypes(include=[np.number]).columns
    total_infs = np.isinf(train_final_clean[numeric_cols]).sum().sum()
    quality_checks['total_infinites'] = total_infs

    # 3. Check for negative sales (should be minimal and expected)
    negative_sales = (train_final_clean['Weekly_Sales'] < 0).sum()
    quality_checks['negative_sales_count'] = negative_sales

    # 4. Check for duplicates
    duplicates = train_final_clean.duplicated(subset=['Store', 'Dept', 'Date']).sum()
    quality_checks['duplicate_rows'] = duplicates

    # 5. Check date consistency
    date_issues = train_final_clean['Date'].isnull().sum()
    quality_checks['date_issues'] = date_issues

    # 6. Check feature value ranges
    sales_stats = {
        'min_sales': train_final_clean['Weekly_Sales'].min(),
        'max_sales': train_final_clean['Weekly_Sales'].max(),
        'mean_sales': train_final_clean['Weekly_Sales'].mean(),
        'std_sales': train_final_clean['Weekly_Sales'].std()
    }

    # Log all quality checks
    for check, value in quality_checks.items():
        mlflow.log_metric(f"quality_{check}", value)

    for stat, value in sales_stats.items():
        mlflow.log_metric(f"sales_{stat}", value)

    print(f"📊 Data Quality Report:")
    print(f"   ✅ Total NaNs: {total_nans}")
    print(f"   ✅ Infinite values: {total_infs}")
    print(f"   ✅ Negative sales: {negative_sales:,} ({negative_sales/len(train_final_clean)*100:.2f}%)")
    print(f"   ✅ Duplicate rows: {duplicates}")
    print(f"   ✅ Date issues: {date_issues}")

    print(f"\n📈 Sales Statistics:")
    print(f"   Min: ${sales_stats['min_sales']:,.2f}")
    print(f"   Max: ${sales_stats['max_sales']:,.2f}")
    print(f"   Mean: ${sales_stats['mean_sales']:,.2f}")
    print(f"   Std: ${sales_stats['std_sales']:,.2f}")

    # Overall assessment
    critical_issues = total_nans + total_infs + duplicates + date_issues

    if critical_issues == 0:
        print(f"\n🎉 EXCELLENT: No critical data quality issues found!")
        print(f"📦 Dataset is ready for Prophet modeling!")
        mlflow.log_param("data_quality_status", "excellent")
    else:
        print(f"\n⚠️  Warning: {critical_issues} critical issues found")
        mlflow.log_param("data_quality_status", "needs_attention")

    # Sample of final dataset
    print(f"\n📋 Sample of cleaned dataset:")
    sample_cols = ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday',
                   'Weekly_Sales_lag_1', 'Weekly_Sales_rolling_mean_4', 'Weekly_Sales_ewm_0.3']
    available_cols = [col for col in sample_cols if col in train_final_clean.columns]
    print(train_final_clean[available_cols].head(10).to_string(index=False))

🔧 Starting final NaN cleanup: 5f09cfc2865b4e409ae582047bb6db43
🔍 Cleaning 6 lag features...
   Fixing Weekly_Sales_lag_1: 37 NaNs
      Before: 37, After: 0
   Fixing Weekly_Sales_lag_2: 97 NaNs
      Before: 97, After: 0
   Fixing Weekly_Sales_lag_3: 154 NaNs
      Before: 154, After: 0
   Fixing Weekly_Sales_lag_4: 226 NaNs
      Before: 226, After: 0
   Fixing Weekly_Sales_lag_8: 526 NaNs
      Before: 526, After: 0
   Fixing Weekly_Sales_lag_12: 995 NaNs
      Before: 995, After: 0

✅ Final cleanup completed!
📊 Dataset shape: (421570, 73)
🔧 Total NaNs remaining: 0
🎉 SUCCESS: Dataset is now completely clean with 0 NaNs!

📋 Final Dataset Summary:
   Shape: (421570, 73)
   Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
   Stores: 45
   Departments: 81
   Store-Dept combinations: 3331
   Average weeks per combination: 126.6

🎯 Dataset is ready for Prophet training!
🏃 View run Final_NaN_Cleanup_Lag_Features at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-

In [19]:
# Step: Holiday-Aware Outlier Detection and Filtering
with mlflow.start_run(run_name="Holiday_Aware_Outlier_Detection") as run:

    print(f"🎄 Starting holiday-aware outlier detection: {run.info.run_id}")

    mlflow.log_param("outlier_detection_method", "holiday_aware_separate_thresholds")
    mlflow.log_param("original_dataset_shape", f"{train_final_clean.shape[0]}x{train_final_clean.shape[1]}")

    # Create copy for outlier analysis
    train_holiday_analysis = train_final_clean.copy()

    # 1. Separate Holiday vs Non-Holiday Data
    print("🔍 Analyzing Holiday vs Non-Holiday sales patterns...")

    holiday_data = train_holiday_analysis[train_holiday_analysis['IsHoliday'] == True]
    non_holiday_data = train_holiday_analysis[train_holiday_analysis['IsHoliday'] == False]

    print(f"   Holiday weeks: {len(holiday_data):,} records ({len(holiday_data)/len(train_holiday_analysis)*100:.1f}%)")
    print(f"   Non-holiday weeks: {len(non_holiday_data):,} records ({len(non_holiday_data)/len(train_holiday_analysis)*100:.1f}%)")

    mlflow.log_metric("holiday_records", len(holiday_data))
    mlflow.log_metric("non_holiday_records", len(non_holiday_data))

    # 2. Statistical Analysis for Each Group
    def calculate_stats(data, label):
        stats = {
            'count': len(data),
            'min': data['Weekly_Sales'].min(),
            'max': data['Weekly_Sales'].max(),
            'mean': data['Weekly_Sales'].mean(),
            'median': data['Weekly_Sales'].median(),
            'std': data['Weekly_Sales'].std(),
            'q1': data['Weekly_Sales'].quantile(0.25),
            'q3': data['Weekly_Sales'].quantile(0.75),
            'q95': data['Weekly_Sales'].quantile(0.95),
            'q99': data['Weekly_Sales'].quantile(0.99),
            'negative_count': (data['Weekly_Sales'] < 0).sum()
        }
        stats['iqr'] = stats['q3'] - stats['q1']

        print(f"\n📊 {label} Sales Statistics:")
        print(f"   Count: {stats['count']:,}")
        print(f"   Mean: ${stats['mean']:,.2f}")
        print(f"   Median: ${stats['median']:,.2f}")
        print(f"   Std: ${stats['std']:,.2f}")
        print(f"   Min: ${stats['min']:,.2f}")
        print(f"   Max: ${stats['max']:,.2f}")
        print(f"   95th percentile: ${stats['q95']:,.2f}")
        print(f"   99th percentile: ${stats['q99']:,.2f}")
        print(f"   Negative sales: {stats['negative_count']:,}")

        # Log to MLflow
        for key, value in stats.items():
            mlflow.log_metric(f"{label.lower()}_{key}", value)

        return stats

    holiday_stats = calculate_stats(holiday_data, "Holiday")
    non_holiday_stats = calculate_stats(non_holiday_data, "Non-Holiday")

    # 3. Compare Holiday vs Non-Holiday patterns
    print(f"\n🔍 Holiday vs Non-Holiday Comparison:")
    mean_ratio = holiday_stats['mean'] / non_holiday_stats['mean']
    median_ratio = holiday_stats['median'] / non_holiday_stats['median']
    max_ratio = holiday_stats['max'] / non_holiday_stats['max']

    print(f"   Holiday mean is {mean_ratio:.1f}x higher than non-holiday")
    print(f"   Holiday median is {median_ratio:.1f}x higher than non-holiday")
    print(f"   Holiday max is {max_ratio:.1f}x higher than non-holiday")

    mlflow.log_metric("holiday_mean_ratio", mean_ratio)
    mlflow.log_metric("holiday_median_ratio", median_ratio)
    mlflow.log_metric("holiday_max_ratio", max_ratio)

    # 4. Holiday-Specific Outlier Detection
    print(f"\n🎯 Applying separate outlier detection for Holiday vs Non-Holiday...")

    def detect_outliers_by_group(data, stats, label, iqr_multiplier=2.5):
        """Detect outliers within a specific group (holiday/non-holiday)"""

        # Method 1: IQR-based
        lower_bound = stats['q1'] - iqr_multiplier * stats['iqr']
        upper_bound = stats['q3'] + iqr_multiplier * stats['iqr']

        # Method 2: Percentile-based (keep 99.5% of data in this group)
        percentile_lower = data['Weekly_Sales'].quantile(0.0025)  # 0.25%
        percentile_upper = data['Weekly_Sales'].quantile(0.9975)  # 99.75%

        # Method 3: Z-score within group
        z_scores = np.abs((data['Weekly_Sales'] - stats['mean']) / stats['std'])
        z_threshold = 3.5 if label == "Holiday" else 3.0  # More lenient for holidays

        # For negative sales - different thresholds for holidays vs non-holidays
        if label == "Holiday":
            negative_threshold = -15000  # Allow more returns during holidays
        else:
            negative_threshold = -8000   # Stricter for non-holidays

        outlier_flags = {
            'iqr': (data['Weekly_Sales'] < lower_bound) | (data['Weekly_Sales'] > upper_bound),
            'percentile': (data['Weekly_Sales'] < percentile_lower) | (data['Weekly_Sales'] > percentile_upper),
            'z_score': z_scores > z_threshold,
            'extreme_negative': data['Weekly_Sales'] < negative_threshold
        }

        # Count outliers by method
        outlier_counts = {method: flags.sum() for method, flags in outlier_flags.items()}

        print(f"\n   {label} Outlier Detection Results:")
        print(f"   IQR bounds: ${lower_bound:,.0f} to ${upper_bound:,.0f}")
        print(f"   Percentile bounds: ${percentile_lower:,.0f} to ${percentile_upper:,.0f}")
        print(f"   Z-score threshold: {z_threshold}")
        print(f"   Negative threshold: ${negative_threshold:,.0f}")

        for method, count in outlier_counts.items():
            percentage = (count / len(data)) * 100
            print(f"     {method}: {count:,} outliers ({percentage:.2f}%)")
            mlflow.log_metric(f"{label.lower()}_outliers_{method}", count)
            mlflow.log_metric(f"{label.lower()}_outliers_{method}_pct", percentage)

        # Smart combination: Use moderate IQR + extreme negative filtering
        combined_outliers = outlier_flags['iqr'] | outlier_flags['extreme_negative']

        return combined_outliers, {
            'bounds': (lower_bound, upper_bound),
            'negative_threshold': negative_threshold,
            'outlier_counts': outlier_counts,
            'combined_count': combined_outliers.sum()
        }

    # Apply outlier detection to each group
    holiday_outliers, holiday_info = detect_outliers_by_group(
        holiday_data, holiday_stats, "Holiday", iqr_multiplier=3.0  # More lenient for holidays
    )

    non_holiday_outliers, non_holiday_info = detect_outliers_by_group(
        non_holiday_data, non_holiday_stats, "Non-Holiday", iqr_multiplier=2.5  # Standard for non-holidays
    )

    # 5. Store-Department Specific Analysis (Holiday-Aware)
    print(f"\n🏪 Store-Department specific outlier detection (holiday-aware)...")

    def detect_store_dept_outliers_holiday_aware(group, multiplier_holiday=3.5, multiplier_non_holiday=2.5):
        """Detect outliers within each store-department, considering holiday status"""
        if len(group) < 10:  # Skip if too few observations
            return pd.Series([False] * len(group), index=group.index)

        outliers = pd.Series([False] * len(group), index=group.index)

        # Separate holiday and non-holiday within this store-dept
        holiday_mask = group['IsHoliday'] == True
        non_holiday_mask = group['IsHoliday'] == False

        # Holiday outliers within this store-dept
        if holiday_mask.sum() >= 3:  # Need at least 3 holiday observations
            holiday_group = group[holiday_mask]['Weekly_Sales']
            q1_h = holiday_group.quantile(0.25)
            q3_h = holiday_group.quantile(0.75)
            iqr_h = q3_h - q1_h

            if iqr_h > 0:  # Avoid division by zero
                lower_h = q1_h - multiplier_holiday * iqr_h
                upper_h = q3_h + multiplier_holiday * iqr_h
                holiday_outliers = (holiday_group < lower_h) | (holiday_group > upper_h)
                outliers[holiday_outliers.index] = holiday_outliers

        # Non-holiday outliers within this store-dept
        if non_holiday_mask.sum() >= 5:  # Need at least 5 non-holiday observations
            non_holiday_group = group[non_holiday_mask]['Weekly_Sales']
            q1_nh = non_holiday_group.quantile(0.25)
            q3_nh = non_holiday_group.quantile(0.75)
            iqr_nh = q3_nh - q1_nh

            if iqr_nh > 0:  # Avoid division by zero
                lower_nh = q1_nh - multiplier_non_holiday * iqr_nh
                upper_nh = q3_nh + multiplier_non_holiday * iqr_nh
                non_holiday_outliers = (non_holiday_group < lower_nh) | (non_holiday_group > upper_nh)
                outliers[non_holiday_outliers.index] = non_holiday_outliers

        return outliers

    # Apply store-department specific detection
    store_dept_outliers = train_holiday_analysis.groupby(['Store', 'Dept']).apply(
        detect_store_dept_outliers_holiday_aware
    ).reset_index(drop=True)

    store_dept_outlier_count = store_dept_outliers.sum()
    print(f"   Store-Dept specific outliers: {store_dept_outlier_count:,} ({store_dept_outlier_count/len(train_holiday_analysis)*100:.2f}%)")
    mlflow.log_metric("store_dept_outliers", store_dept_outlier_count)

    # 6. Combine Results and Apply Filtering
    print(f"\n🧠 Combining results and applying smart filtering...")

    # Create outlier mask for the full dataset
    full_outlier_mask = pd.Series([False] * len(train_holiday_analysis), index=train_holiday_analysis.index)

    # Apply holiday-specific outliers
    holiday_indices = train_holiday_analysis[train_holiday_analysis['IsHoliday'] == True].index
    full_outlier_mask[holiday_indices] = holiday_outliers.values

    # Apply non-holiday-specific outliers
    non_holiday_indices = train_holiday_analysis[train_holiday_analysis['IsHoliday'] == False].index
    full_outlier_mask[non_holiday_indices] = non_holiday_outliers.values

    # Optional: Also include store-department specific outliers
    # full_outlier_mask = full_outlier_mask | store_dept_outliers

    # Apply filtering
    train_holiday_filtered = train_holiday_analysis[~full_outlier_mask].copy()

    # 7. Results Summary
    original_count = len(train_holiday_analysis)
    filtered_count = len(train_holiday_filtered)
    removed_count = original_count - filtered_count
    removal_percentage = (removed_count / original_count) * 100

    # Holiday-specific removal stats
    holiday_removed = holiday_outliers.sum()
    non_holiday_removed = non_holiday_outliers.sum()
    holiday_removal_rate = (holiday_removed / len(holiday_data)) * 100
    non_holiday_removal_rate = (non_holiday_removed / len(non_holiday_data)) * 100

    print(f"\n✅ Holiday-aware filtering completed!")
    print(f"   Original rows: {original_count:,}")
    print(f"   Filtered rows: {filtered_count:,}")
    print(f"   Total removed: {removed_count:,} ({removal_percentage:.2f}%)")
    print(f"   ")
    print(f"   Holiday outliers removed: {holiday_removed:,} ({holiday_removal_rate:.2f}% of holiday data)")
    print(f"   Non-holiday outliers removed: {non_holiday_removed:,} ({non_holiday_removal_rate:.2f}% of non-holiday data)")

    # Log final metrics
    mlflow.log_metric("total_removed", removed_count)
    mlflow.log_metric("removal_percentage", removal_percentage)
    mlflow.log_metric("holiday_removed", holiday_removed)
    mlflow.log_metric("non_holiday_removed", non_holiday_removed)
    mlflow.log_metric("holiday_removal_rate", holiday_removal_rate)
    mlflow.log_metric("non_holiday_removal_rate", non_holiday_removal_rate)

    # 8. Final dataset verification
    print(f"\n📊 Final filtered dataset analysis:")
    final_holiday_data = train_holiday_filtered[train_holiday_filtered['IsHoliday'] == True]
    final_non_holiday_data = train_holiday_filtered[train_holiday_filtered['IsHoliday'] == False]

    print(f"   Holiday weeks remaining: {len(final_holiday_data):,}")
    print(f"   Non-holiday weeks remaining: {len(final_non_holiday_data):,}")
    print(f"   Holiday mean sales: ${final_holiday_data['Weekly_Sales'].mean():,.2f}")
    print(f"   Non-holiday mean sales: ${final_non_holiday_data['Weekly_Sales'].mean():,.2f}")
    print(f"   Holiday/Non-holiday ratio: {final_holiday_data['Weekly_Sales'].mean() / final_non_holiday_data['Weekly_Sales'].mean():.1f}x")

    print(f"\n🎉 Holiday-aware outlier filtering completed!")
    print(f"📊 Final dataset shape: {train_holiday_filtered.shape}")
    print(f"🎯 Dataset preserves holiday sales patterns and is ready for Prophet training!")

    # Save detailed summary
    holiday_aware_summary = {
        'original_shape': train_holiday_analysis.shape,
        'filtered_shape': train_holiday_filtered.shape,
        'holiday_stats': holiday_stats,
        'non_holiday_stats': non_holiday_stats,
        'removal_summary': {
            'total_removed': removed_count,
            'holiday_removed': holiday_removed,
            'non_holiday_removed': non_holiday_removed,
            'removal_rates': {
                'overall': removal_percentage,
                'holiday': holiday_removal_rate,
                'non_holiday': non_holiday_removal_rate
            }
        },
        'filtering_parameters': {
            'holiday_iqr_multiplier': 3.0,
            'non_holiday_iqr_multiplier': 2.5,
            'holiday_negative_threshold': holiday_info['negative_threshold'],
            'non_holiday_negative_threshold': non_holiday_info['negative_threshold']
        }
    }

    mlflow.log_dict(holiday_aware_summary, "holiday_aware_outlier_summary.json")

# End MLflow run
mlflow.end_run()

🎄 Starting holiday-aware outlier detection: 608ea81cd34e48829db30c91487f2c08
🔍 Analyzing Holiday vs Non-Holiday sales patterns...
   Holiday weeks: 29,661 records (7.0%)
   Non-holiday weeks: 391,909 records (93.0%)

📊 Holiday Sales Statistics:
   Count: 29,661
   Mean: $17,035.82
   Median: $7,947.74
   Std: $27,222.00
   Min: $-798.00
   Max: $693,099.36
   95th percentile: $63,916.21
   99th percentile: $118,773.24
   Negative sales: 98

📊 Non-Holiday Sales Statistics:
   Count: 391,909
   Mean: $15,901.45
   Median: $7,589.95
   Std: $22,330.75
   Min: $-4,988.94
   Max: $406,988.63
   95th percentile: $61,018.50
   99th percentile: $105,704.59
   Negative sales: 1,187

🔍 Holiday vs Non-Holiday Comparison:
   Holiday mean is 1.1x higher than non-holiday
   Holiday median is 1.0x higher than non-holiday
   Holiday max is 1.7x higher than non-holiday

🎯 Applying separate outlier detection for Holiday vs Non-Holiday...

   Holiday Outlier Detection Results:
   IQR bounds: $-55,254 to 

In [20]:
# Step: Time Series Data Splitting (80/20)
with mlflow.start_run(run_name="Time_Series_Data_Split_80_20") as run:

    print(f"⏰ Starting time series data split: {run.info.run_id}")

    mlflow.log_param("split_method", "temporal_80_20")
    mlflow.log_param("original_data_shape", f"{train_holiday_filtered.shape[0]}x{train_holiday_filtered.shape[1]}")

    # 1. Prepare and sort data
    print("📊 Preparing data for time series split...")

    # Create a copy and sort by date
    df_split = train_holiday_filtered.copy()
    df_split = df_split.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

    print(f"   Dataset shape: {df_split.shape}")
    print(f"   Date range: {df_split['Date'].min()} to {df_split['Date'].max()}")
    print(f"   Total records: {len(df_split):,}")

    # 2. Analyze date distribution
    print(f"\n📅 Analyzing date distribution...")

    # Get unique dates and sort them
    unique_dates = sorted(df_split['Date'].unique())
    total_dates = len(unique_dates)

    print(f"   Unique dates: {total_dates}")
    print(f"   First date: {unique_dates[0]}")
    print(f"   Last date: {unique_dates[-1]}")
    print(f"   Date span: {(unique_dates[-1] - unique_dates[0]).days} days")

    # Records per date
    records_per_date = df_split['Date'].value_counts().sort_index()
    print(f"   Average records per date: {records_per_date.mean():.1f}")
    print(f"   Min records per date: {records_per_date.min()}")
    print(f"   Max records per date: {records_per_date.max()}")

    # 3. Calculate split point (80% of time period)
    print(f"\n🔪 Calculating 80/20 time split...")

    split_idx = int(total_dates * 0.8)
    split_date = unique_dates[split_idx]

    print(f"   Split index: {split_idx} out of {total_dates} dates")
    print(f"   Split date: {split_date}")
    print(f"   ")
    print(f"   Training period: {unique_dates[0]} to {unique_dates[split_idx-1]}")
    print(f"   Validation period: {split_date} to {unique_dates[-1]}")

    # 4. Create train/validation splits
    print(f"\n✂️ Creating train/validation splits...")

    train_mask = df_split['Date'] < split_date
    val_mask = df_split['Date'] >= split_date

    train_data = df_split[train_mask].copy()
    val_data = df_split[val_mask].copy()

    # 5. Split analysis
    print(f"\n📊 Split Analysis:")
    print(f"=" * 60)

    # Basic counts
    print(f"Training Set:")
    print(f"   Records: {len(train_data):,} ({len(train_data)/len(df_split)*100:.1f}%)")
    print(f"   Date range: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"   Unique dates: {train_data['Date'].nunique()}")
    print(f"   Stores: {train_data['Store'].nunique()}")
    print(f"   Departments: {train_data['Dept'].nunique()}")
    print(f"   Store-Dept combinations: {train_data.groupby(['Store', 'Dept']).ngroups}")

    print(f"\nValidation Set:")
    print(f"   Records: {len(val_data):,} ({len(val_data)/len(df_split)*100:.1f}%)")
    print(f"   Date range: {val_data['Date'].min()} to {val_data['Date'].max()}")
    print(f"   Unique dates: {val_data['Date'].nunique()}")
    print(f"   Stores: {val_data['Store'].nunique()}")
    print(f"   Departments: {val_data['Dept'].nunique()}")
    print(f"   Store-Dept combinations: {val_data.groupby(['Store', 'Dept']).ngroups}")

    # 6. Sales statistics comparison
    print(f"\n💰 Sales Statistics Comparison:")
    print(f"=" * 60)

    train_sales_stats = {
        'count': len(train_data),
        'mean': train_data['Weekly_Sales'].mean(),
        'median': train_data['Weekly_Sales'].median(),
        'std': train_data['Weekly_Sales'].std(),
        'min': train_data['Weekly_Sales'].min(),
        'max': train_data['Weekly_Sales'].max(),
        'negative_count': (train_data['Weekly_Sales'] < 0).sum()
    }

    val_sales_stats = {
        'count': len(val_data),
        'mean': val_data['Weekly_Sales'].mean(),
        'median': val_data['Weekly_Sales'].median(),
        'std': val_data['Weekly_Sales'].std(),
        'min': val_data['Weekly_Sales'].min(),
        'max': val_data['Weekly_Sales'].max(),
        'negative_count': (val_data['Weekly_Sales'] < 0).sum()
    }

    print(f"Training Sales Statistics:")
    print(f"   Count: {train_sales_stats['count']:,}")
    print(f"   Mean: ${train_sales_stats['mean']:,.2f}")
    print(f"   Median: ${train_sales_stats['median']:,.2f}")
    print(f"   Std: ${train_sales_stats['std']:,.2f}")
    print(f"   Min: ${train_sales_stats['min']:,.2f}")
    print(f"   Max: ${train_sales_stats['max']:,.2f}")
    print(f"   Negative sales: {train_sales_stats['negative_count']:,}")

    print(f"\nValidation Sales Statistics:")
    print(f"   Count: {val_sales_stats['count']:,}")
    print(f"   Mean: ${val_sales_stats['mean']:,.2f}")
    print(f"   Median: ${val_sales_stats['median']:,.2f}")
    print(f"   Std: ${val_sales_stats['std']:,.2f}")
    print(f"   Min: ${val_sales_stats['min']:,.2f}")
    print(f"   Max: ${val_sales_stats['max']:,.2f}")
    print(f"   Negative sales: {val_sales_stats['negative_count']:,}")

    # 7. Holiday distribution
    print(f"\n🎄 Holiday Distribution:")
    print(f"=" * 60)

    train_holidays = train_data['IsHoliday'].value_counts()
    val_holidays = val_data['IsHoliday'].value_counts()

    print(f"Training Set Holidays:")
    print(f"   Holiday weeks: {train_holidays.get(True, 0):,}")
    print(f"   Non-holiday weeks: {train_holidays.get(False, 0):,}")
    print(f"   Holiday percentage: {(train_holidays.get(True, 0)/len(train_data)*100):.2f}%")

    print(f"\nValidation Set Holidays:")
    print(f"   Holiday weeks: {val_holidays.get(True, 0):,}")
    print(f"   Non-holiday weeks: {val_holidays.get(False, 0):,}")
    print(f"   Holiday percentage: {(val_holidays.get(True, 0)/len(val_data)*100):.2f}%")

    # 8. Store and Department coverage
    print(f"\n🏪 Store and Department Coverage:")
    print(f"=" * 60)

    # Check if all stores/departments are in both sets
    train_stores = set(train_data['Store'].unique())
    val_stores = set(val_data['Store'].unique())
    train_depts = set(train_data['Dept'].unique())
    val_depts = set(val_data['Dept'].unique())

    missing_stores_in_val = train_stores - val_stores
    missing_depts_in_val = train_depts - val_depts

    print(f"Store Coverage:")
    print(f"   Training stores: {len(train_stores)}")
    print(f"   Validation stores: {len(val_stores)}")
    print(f"   Missing stores in validation: {len(missing_stores_in_val)}")
    if missing_stores_in_val:
        print(f"     Missing stores: {sorted(missing_stores_in_val)}")

    print(f"\nDepartment Coverage:")
    print(f"   Training departments: {len(train_depts)}")
    print(f"   Validation departments: {len(val_depts)}")
    print(f"   Missing departments in validation: {len(missing_depts_in_val)}")
    if missing_depts_in_val:
        print(f"     Missing departments: {sorted(missing_depts_in_val)}")

    # 9. Time series continuity check
    print(f"\n⏰ Time Series Continuity Check:")
    print(f"=" * 60)

    # Check for gaps in time series
    def check_time_gaps(data, name):
        data_sorted = data.sort_values('Date')
        date_diffs = data_sorted['Date'].diff().dropna()

        # Expected difference is 7 days (weekly data)
        expected_diff = pd.Timedelta(days=7)
        gaps = date_diffs[date_diffs != expected_diff]

        print(f"{name} Time Gaps:")
        print(f"   Expected interval: 7 days")
        print(f"   Actual intervals found: {date_diffs.unique()}")
        print(f"   Number of gaps: {len(gaps)}")
        if len(gaps) > 0:
            print(f"   Gap sizes: {gaps.unique()}")

    check_time_gaps(train_data, "Training")
    check_time_gaps(val_data, "Validation")

    # 10. Sample data preview
    print(f"\n📋 Sample Data Preview:")
    print(f"=" * 60)

    print(f"Training Data Sample (first 5 rows):")
    print(train_data[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].head())

    print(f"\nValidation Data Sample (first 5 rows):")
    print(val_data[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].head())

    # 11. Log metrics to MLflow
    mlflow.log_param("split_date", str(split_date))
    mlflow.log_param("train_start_date", str(train_data['Date'].min()))
    mlflow.log_param("train_end_date", str(train_data['Date'].max()))
    mlflow.log_param("val_start_date", str(val_data['Date'].min()))
    mlflow.log_param("val_end_date", str(val_data['Date'].max()))

    mlflow.log_metric("train_records", len(train_data))
    mlflow.log_metric("val_records", len(val_data))
    mlflow.log_metric("train_percentage", len(train_data)/len(df_split)*100)
    mlflow.log_metric("val_percentage", len(val_data)/len(df_split)*100)

    mlflow.log_metric("train_mean_sales", train_sales_stats['mean'])
    mlflow.log_metric("val_mean_sales", val_sales_stats['mean'])
    mlflow.log_metric("train_holiday_count", train_holidays.get(True, 0))
    mlflow.log_metric("val_holiday_count", val_holidays.get(True, 0))

    # 12. Save split datasets for later use
    print(f"\n💾 Saving split datasets...")

    # Save as variables for next steps
    train_final = train_data.copy()
    val_final = val_data.copy()

    print(f"   ✅ Training data saved: {len(train_final):,} records")
    print(f"   ✅ Validation data saved: {len(val_final):,} records")

    # Save split summary
    split_summary = {
        'split_method': 'temporal_80_20',
        'split_date': str(split_date),
        'original_data_shape': list(df_split.shape),
        'train_data_shape': list(train_data.shape),
        'val_data_shape': list(val_data.shape),
        'train_date_range': [str(train_data['Date'].min()), str(train_data['Date'].max())],
        'val_date_range': [str(val_data['Date'].min()), str(val_data['Date'].max())],
        'train_sales_stats': train_sales_stats,
        'val_sales_stats': val_sales_stats,
        'coverage_check': {
            'missing_stores_in_val': list(missing_stores_in_val),
            'missing_depts_in_val': list(missing_depts_in_val)
        }
    }

    mlflow.log_dict(split_summary, "split_summary.json")

    print(f"\n🎉 Time Series Split Completed Successfully!")
    print(f"📊 Ready for model training!")
    print(f"⚠️  Note: This is a proper time series split - no future data leakage")

# End MLflow run
mlflow.end_run()

print(f"\n" + "="*80)
print(f"📊 SPLIT SUMMARY")
print(f"="*80)
print(f"✅ Training Set: {len(train_final):,} records")
print(f"✅ Validation Set: {len(val_final):,} records")
print(f"📅 Split Date: {split_date}")
print(f"🎯 Ready for XGBoost training!")
print(f"="*80)

⏰ Starting time series data split: f9fc0fa0a73a4048be3b3b374a5ea53b
📊 Preparing data for time series split...
   Dataset shape: (403911, 73)
   Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
   Total records: 403,911

📅 Analyzing date distribution...
   Unique dates: 143
   First date: 2010-02-05 00:00:00
   Last date: 2012-10-26 00:00:00
   Date span: 994 days
   Average records per date: 2824.6
   Min records per date: 2627
   Max records per date: 2941

🔪 Calculating 80/20 time split...
   Split index: 114 out of 143 dates
   Split date: 2012-04-13 00:00:00
   
   Training period: 2010-02-05 00:00:00 to 2012-04-06 00:00:00
   Validation period: 2012-04-13 00:00:00 to 2012-10-26 00:00:00

✂️ Creating train/validation splits...

📊 Split Analysis:
Training Set:
   Records: 321,636 (79.6%)
   Date range: 2010-02-05 00:00:00 to 2012-04-06 00:00:00
   Unique dates: 114
   Stores: 45
   Departments: 81
   Store-Dept combinations: 3279

Validation Set:
   Records: 82,275 (20.4%)
   

In [24]:
# Step: XGBoost Training with Data Type Fix
with mlflow.start_run(run_name="XGBoost_Training_DataType_Fixed") as run:

    print(f"🚀 Starting XGBoost training with data type fix: {run.info.run_id}")

    mlflow.log_param("model_type", "xgboost_regressor")
    mlflow.log_param("evaluation_metric", "WMAE_weighted_mean_absolute_error")
    mlflow.log_param("train_size", len(train_final))
    mlflow.log_param("val_size", len(val_final))

    # Import required libraries
    import xgboost as xgb
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.preprocessing import LabelEncoder
    import joblib

    # 1. Prepare data with proper data types
    print("🔧 Preparing data with proper data types...")

    # Define target and features
    target_column = 'Weekly_Sales'

    # Exclude non-feature columns
    exclude_columns = [
        'Weekly_Sales',  # target
        'Date',         # original date (we have date features)
        'Store',        # will encode separately
        'Dept',         # will encode separately
    ]

    # Get feature columns
    feature_columns = [col for col in train_final.columns if col not in exclude_columns]

    print(f"   Total features available: {len(feature_columns)}")

    # 2. Handle categorical encoding and data types
    print(f"🏷️ Encoding categorical variables and fixing data types...")

    # Create copies for encoding
    train_encoded = train_final.copy()
    val_encoded = val_final.copy()

    # Encode Store and Dept
    categorical_features = ['Store', 'Dept']
    label_encoders = {}

    for cat_feature in categorical_features:
        le = LabelEncoder()
        # Fit on training data only
        train_encoded[f'{cat_feature}_encoded'] = le.fit_transform(train_encoded[cat_feature])
        # Transform validation data
        val_encoded[f'{cat_feature}_encoded'] = le.transform(val_encoded[cat_feature])
        label_encoders[cat_feature] = le

        # Add encoded feature to feature list
        if f'{cat_feature}_encoded' not in feature_columns:
            feature_columns.append(f'{cat_feature}_encoded')

        print(f"   Encoded {cat_feature}: {train_encoded[cat_feature].nunique()} unique values")

    print(f"   Final feature count: {len(feature_columns)}")

    # 3. Prepare X and y with proper data types
    X_train = train_encoded[feature_columns].copy()
    y_train = train_encoded[target_column].copy()
    X_val = val_encoded[feature_columns].copy()
    y_val = val_encoded[target_column].copy()

    # 4. Fix data types - Convert all to numeric
    print(f"🔧 Converting all features to numeric types...")

    # Check current data types
    print(f"   Checking data types before conversion...")
    object_columns = []
    for col in X_train.columns:
        if X_train[col].dtype == 'object':
            object_columns.append(col)

    if object_columns:
        print(f"   Found {len(object_columns)} object columns: {object_columns}")

        # Convert object columns to numeric
        for col in object_columns:
            try:
                # Try to convert to numeric
                X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
                X_val[col] = pd.to_numeric(X_val[col], errors='coerce')
                print(f"     Converted {col} to numeric")
            except Exception as e:
                print(f"     Warning: Could not convert {col} to numeric: {e}")
    else:
        print(f"   ✅ No object columns found")

    # Convert boolean columns to int
    bool_columns = []
    for col in X_train.columns:
        if X_train[col].dtype == 'bool':
            bool_columns.append(col)

    if bool_columns:
        print(f"   Found {len(bool_columns)} boolean columns, converting to int...")
        for col in bool_columns:
            X_train[col] = X_train[col].astype(int)
            X_val[col] = X_val[col].astype(int)
            print(f"     Converted {col} from bool to int")

    # Ensure all columns are float64 or int64
    print(f"   Converting all features to float64...")
    for col in X_train.columns:
        if X_train[col].dtype not in ['float64', 'int64', 'float32', 'int32']:
            X_train[col] = X_train[col].astype('float64')
            X_val[col] = X_val[col].astype('float64')

    # Handle any missing values that might have been introduced during conversion
    print(f"🔍 Checking for missing values after conversion...")
    train_missing = X_train.isnull().sum().sum()
    val_missing = X_val.isnull().sum().sum()

    if train_missing > 0 or val_missing > 0:
        print(f"   Warning: Missing values found after conversion - Train: {train_missing}, Val: {val_missing}")
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)
        print(f"   ✅ Missing values filled with 0")
    else:
        print(f"   ✅ No missing values found")

    # Final data type check
    print(f"\n📊 Final data type verification:")
    print(f"   X_train dtypes: {X_train.dtypes.value_counts().to_dict()}")
    print(f"   X_val dtypes: {X_val.dtypes.value_counts().to_dict()}")

    print(f"\n📊 Final dataset shapes:")
    print(f"   X_train: {X_train.shape}")
    print(f"   y_train: {y_train.shape}")
    print(f"   X_val: {X_val.shape}")
    print(f"   y_val: {y_val.shape}")

    # 5. Define WMAE function and weights
    print(f"⚖️ Setting up WMAE (Weighted Mean Absolute Error)...")

    def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
        """
        Calculate Weighted Mean Absolute Error
        Holiday weeks are weighted 5x higher than non-holiday weeks
        """
        # Calculate absolute errors
        abs_errors = np.abs(y_true - y_pred)

        # Create weights: 5.0 for holiday weeks, 1.0 for non-holiday weeks
        weights = np.where(is_holiday, holiday_weight, 1.0)

        # Calculate weighted mean absolute error
        wmae = np.sum(weights * abs_errors) / np.sum(weights)

        return wmae

    def calculate_comprehensive_metrics(y_true, y_pred, is_holiday, dataset_name):
        """Calculate comprehensive metrics including WMAE"""

        # Standard metrics
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        mape = np.mean(np.abs((y_true - y_pred) / np.abs(y_true))) * 100

        # WMAE (main metric for Walmart competition)
        wmae = calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0)

        # Holiday vs Non-holiday breakdown
        holiday_mask = is_holiday == True
        non_holiday_mask = is_holiday == False

        if holiday_mask.sum() > 0:
            holiday_mae = mean_absolute_error(y_true[holiday_mask], y_pred[holiday_mask])
            holiday_count = holiday_mask.sum()
        else:
            holiday_mae = 0
            holiday_count = 0

        if non_holiday_mask.sum() > 0:
            non_holiday_mae = mean_absolute_error(y_true[non_holiday_mask], y_pred[non_holiday_mask])
            non_holiday_count = non_holiday_mask.sum()
        else:
            non_holiday_mae = 0
            non_holiday_count = 0

        metrics = {
            'wmae': wmae,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'holiday_mae': holiday_mae,
            'non_holiday_mae': non_holiday_mae,
            'holiday_count': holiday_count,
            'non_holiday_count': non_holiday_count,
            'mean_true': np.mean(y_true),
            'mean_pred': np.mean(y_pred)
        }

        print(f"\n📊 {dataset_name} Metrics:")
        print(f"   🎯 WMAE (Main Metric): ${metrics['wmae']:,.2f}")
        print(f"   📊 MAE: ${metrics['mae']:,.2f}")
        print(f"   📊 RMSE: ${metrics['rmse']:,.2f}")
        print(f"   📊 R²: {metrics['r2']:.4f}")
        print(f"   📊 MAPE: {metrics['mape']:.2f}%")
        print(f"   🎄 Holiday MAE: ${metrics['holiday_mae']:,.2f} ({metrics['holiday_count']} weeks)")
        print(f"   📅 Non-Holiday MAE: ${metrics['non_holiday_mae']:,.2f} ({metrics['non_holiday_count']} weeks)")
        print(f"   💰 Mean True: ${metrics['mean_true']:,.2f}")
        print(f"   💰 Mean Pred: ${metrics['mean_pred']:,.2f}")

        # Log to MLflow
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{dataset_name.lower()}_{metric_name}", value)

        return metrics

    # 6. XGBoost model configuration
    print(f"\n🎯 Configuring XGBoost model...")

    # XGBoost parameters
    xgb_params = {
        # Objective
        'objective': 'reg:squarederror',

        # Tree structure
        'max_depth': 8,
        'min_child_weight': 3,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'colsample_bylevel': 0.8,

        # Learning
        'learning_rate': 0.1,
        'n_estimators': 1000,

        # Regularization
        'reg_alpha': 0.1,
        'reg_lambda': 1.0,

        # Performance
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 1
    }

    # Log parameters
    for key, value in xgb_params.items():
        mlflow.log_param(f"xgb_{key}", value)

    print(f"   Parameters configured:")
    for key, value in xgb_params.items():
        print(f"     {key}: {value}")

    # 7. Train XGBoost model
    print(f"\n🚀 Training XGBoost model...")

    model = xgb.XGBRegressor(**xgb_params)

    # Simple training without early stopping for now
    model.fit(X_train, y_train, verbose=100)

    print(f"\n✅ Training completed!")
    print(f"   Total estimators used: {model.n_estimators}")

    mlflow.log_param("total_estimators", model.n_estimators)

    # 8. Make predictions
    print(f"\n🔮 Making predictions...")

    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    print(f"   Training predictions: {len(y_train_pred):,}")
    print(f"   Validation predictions: {len(y_val_pred):,}")

    # 9. Evaluate with WMAE and other metrics
    print(f"\n📈 Evaluating model performance...")

    # Get holiday information for WMAE calculation
    train_holiday_flags = train_encoded['IsHoliday'].values
    val_holiday_flags = val_encoded['IsHoliday'].values

    # Calculate comprehensive metrics
    train_metrics = calculate_comprehensive_metrics(
        y_train, y_train_pred, train_holiday_flags, "Training"
    )

    val_metrics = calculate_comprehensive_metrics(
        y_val, y_val_pred, val_holiday_flags, "Validation"
    )

    # 10. Feature importance analysis
    print(f"\n🔍 Analyzing feature importance...")

    feature_importance = model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_columns,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)

    print(f"\n   Top 15 Most Important Features:")
    for i, (_, row) in enumerate(feature_importance_df.head(15).iterrows()):
        print(f"     {i+1:2d}. {row['feature']:<40} {row['importance']:.4f}")

    # Log top features
    top_15_features = feature_importance_df.head(15)
    for i, (_, row) in enumerate(top_15_features.iterrows()):
        mlflow.log_metric(f"feature_importance_rank_{i+1}", row['importance'])
        mlflow.log_param(f"top_feature_{i+1}", row['feature'])

    # 11. Quick error analysis
    print(f"\n🔬 Quick error analysis...")

    # Create predictions dataframe
    val_analysis = val_encoded[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']].copy()
    val_analysis['Predicted'] = y_val_pred
    val_analysis['Error'] = val_analysis['Weekly_Sales'] - val_analysis['Predicted']
    val_analysis['Abs_Error'] = np.abs(val_analysis['Error'])

    # Error statistics
    print(f"\n   Error Distribution:")
    print(f"     Mean Error: ${val_analysis['Error'].mean():,.2f}")
    print(f"     Mean Abs Error: ${val_analysis['Abs_Error'].mean():,.2f}")
    print(f"     Median Abs Error: ${val_analysis['Abs_Error'].median():,.2f}")
    print(f"     Max Abs Error: ${val_analysis['Abs_Error'].max():,.2f}")

    # 12. Save results
    print(f"\n💾 Saving model and results...")

    # Save model
    model_path = "xgboost_walmart_wmae_model.joblib"
    joblib.dump(model, model_path)
    mlflow.log_artifact(model_path)

    # Save feature importance
    feature_importance_df.to_csv("feature_importance.csv", index=False)
    mlflow.log_artifact("feature_importance.csv")

    # Save predictions
    val_analysis.to_csv("validation_predictions_wmae.csv", index=False)
    mlflow.log_artifact("validation_predictions_wmae.csv")

    print(f"   ✅ Model saved: {model_path}")
    print(f"   ✅ Results saved: validation_predictions_wmae.csv")

    # 13. Final summary
    print(f"\n🎉 XGBoost Training Summary:")
    print(f"=" * 80)
    print(f"   Model: XGBoost Regressor")
    print(f"   Training samples: {len(X_train):,}")
    print(f"   Validation samples: {len(X_val):,}")
    print(f"   Features used: {len(feature_columns)}")
    print(f"   Total estimators: {model.n_estimators}")
    print(f"")
    print(f"   🎯 KEY METRICS (Validation Set):")
    print(f"     WMAE (Walmart Competition Metric): ${val_metrics['wmae']:,.2f}")
    print(f"     MAE: ${val_metrics['mae']:,.2f}")
    print(f"     RMSE: ${val_metrics['rmse']:,.2f}")
    print(f"     R²: {val_metrics['r2']:.4f}")
    print(f"")
    print(f"   🎄 Holiday Performance:")
    print(f"     Holiday weeks MAE: ${val_metrics['holiday_mae']:,.2f}")
    print(f"     Non-holiday weeks MAE: ${val_metrics['non_holiday_mae']:,.2f}")
    print(f"     Holiday weight in WMAE: 5x")
    print(f"=" * 80)

    # Create comprehensive summary
    model_summary = {
        'model_type': 'XGBoost',
        'evaluation_focus': 'WMAE_weighted_mean_absolute_error',
        'training_data_size': len(X_train),
        'validation_data_size': len(X_val),
        'num_features': len(feature_columns),
        'validation_metrics': val_metrics,
        'training_metrics': train_metrics,
        'parameters': xgb_params,
        'holiday_weight': 5.0,
        'top_10_features': feature_importance_df.head(10).to_dict('records')
    }

    mlflow.log_dict(model_summary, "xgboost_wmae_summary.json")

# End MLflow run
mlflow.end_run()

print(f"\n🏃 View detailed results at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/")

🚀 Starting XGBoost training with data type fix: 912678876c794318881e59358327994d
🔧 Preparing data with proper data types...
   Total features available: 69
🏷️ Encoding categorical variables and fixing data types...
   Encoded Store: 45 unique values
   Encoded Dept: 81 unique values
   Final feature count: 71
🔧 Converting all features to numeric types...
   Checking data types before conversion...
   Found 1 object columns: ['Type']
     Converted Type to numeric
   Found 1 boolean columns, converting to int...
     Converted IsHoliday from bool to int
   Converting all features to float64...
🔍 Checking for missing values after conversion...
   ✅ Missing values filled with 0

📊 Final data type verification:
   X_train dtypes: {dtype('float64'): 38, dtype('int64'): 26, dtype('int32'): 7}
   X_val dtypes: {dtype('float64'): 38, dtype('int64'): 26, dtype('int32'): 7}

📊 Final dataset shapes:
   X_train: (321636, 71)
   y_train: (321636,)
   X_val: (82275, 71)
   y_val: (82275,)
⚖️ Setting